<h1><center><font color='Black'>Machine Learning</font></center></h1>

In [ ]:
#-------------imports--------------#
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier

import folium
import re
import matplotlib.pyplot as plt
from folium.plugins import HeatMap

cols = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
        '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', 
        '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
        '#000075', '#808080']*10

In [ ]:
data = pd.read_csv('Data/df.csv')
data

In [ ]:
data["Date"] = pd.to_datetime(data['Date'],format= '%Y-%m-%d')


In [ ]:
data['Datetime'] = pd.to_datetime(data['Date'].apply(str)+' '+data['Time'])


We will add columns of the hour and a columns with longitude and latitude of each city

In [ ]:
dfCor = pd.read_csv('Data/alt_long.csv', encoding='utf-8-sig')
data['lat'] = 0
data['long'] = 0
for ind in dfCor.index:
    data['lat'] = data['lat'].mask( data['Zone'].str.contains( dfCor['name'][ind] ) , dfCor['latt'][ind])
    data['long'] = data['long'].mask( data['Zone'].str.contains( dfCor['name'][ind] ) , dfCor['long'][ind])


In [ ]:
data = data[data['lat'] != 0]
data.reset_index(inplace=True)


In [ ]:
#make a new column with the date and time
data['year'] = pd.to_datetime(data['Date']).dt.year
data['month'] = pd.to_datetime(data['Date']).dt.month
data['day'] = pd.to_datetime(data['Date']).dt.day
data['hour'] = pd.to_datetime(data['Time'], format='%H:%M').dt.hour

In [ ]:
data = data.drop(['Date','Time'], axis= 1)

In [ ]:
data = data.rename(columns={'Zone': 'NAME'})
data = data.rename(columns={'lat': 'LAT'})
data = data.rename(columns={'long': 'LON'})


Let's see how its look like in scatter plot 

In [ ]:
X=np.array(data[['LON','LAT','hour']],dtype='float64')
plt.scatter(X[:,0],X[:,1],alpha=0.2,s=50)
plt.grid(True)

**Unsupervised Learning** - Kmean

In [ ]:
plt_map=folium.Map(location=[data.LAT.mean(),data.LON.mean()],zoom_start=5,
             tiles='OpenStreetMap')
for _,row in data.iterrows():
    folium.CircleMarker(
        location=[row.LAT,row.LON],
        radius=5,
        popup=re.sub(r'[^a-zA-Z ]+','',row.NAME),
        color='blue',
        fill=True
    ).add_to(plt_map)
plt_map

In [ ]:
k_range=range(4,15)
kmeans_per_k=[]
for k in k_range:
    kmeans=KMeans(n_clusters=k,random_state=0).fit(X)
    kmeans_per_k.append(kmeans)

In [ ]:
silh_scores=[silhouette_score(X,model.labels_) for model in kmeans_per_k]
best_index = np.argmax(silh_scores)
best_k = k_range[best_index]
best_score = silh_scores[best_index]
print("best k value:",best_k)
print("silhouette score:",best_score)


In [ ]:

plt.figure(figsize=(8, 3))
plt.grid(True)
plt.plot(k_range, silh_scores, "bo-")
plt.xlabel("k", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.plot(best_k, best_score, "rs")
plt.show()

In [ ]:
inertias = [model.inertia_ for model in kmeans_per_k]
best_inertia = inertias[best_index]

plt.figure(figsize=(8, 3.5))
plt.grid(True)
plt.plot(k_range, inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.plot(best_k, best_inertia, "rs")
plt.show()

In [ ]:
k=best_k
model=KMeans(n_clusters=k,random_state=42).fit(X)
pred=model.predict(X)
data[f'CLUSTER_kmeans']=pred

In [ ]:
def create_map(data,cluster_col):
    m = folium.Map(location=[data.LAT.mean(), data.LON.mean()], zoom_start=9, tiles='openstreetmap')

    for _, row in data.iterrows():

        # get a colour
        if row[cluster_col]==-1:
            cluster_colour='black'
        else:
            cluster_colour = cols[row[cluster_col]]

        folium.CircleMarker(
            location=[row.LAT,row.LON],
            radius=5,
            popup= row[cluster_col],
            color=cluster_colour,
            fill=True,
            fill_color=cluster_colour
        ).add_to(m)
    return m

In [ ]:
print(f'Silhouette Score: {silhouette_score(X, pred)}')


In [ ]:

plt_map=create_map(data,'CLUSTER_kmeans')   
plt_map.save('kmeans_map.html')
plt_map

Heat Map, Military operation 5-6/2021

In [ ]:
data2 = data[data['Datetime'].dt.year==2021]
data2 = data2[data2['Datetime'].dt.month.between(5,6)]

#Make the list of Lat an Lng
lat = data2.LAT.tolist()
lng = data2.LON.tolist()

#Create the Map
map = folium.Map(
    location=[data2.LAT.mean(),data2.LON.mean()],
    tiles='OpenStreetMap',
    zoom_start=12
)
# [32.05, 34.80], tel aviv
HeatMap(list(zip(lat, lng))).add_to(map)
map